# 3편: CNN으로 이미지 분류하기

이 노트북에서는 다음을 학습합니다:
- 합성곱(Convolution) 연산의 원리
- CNN의 핵심 구성 요소 (Conv2d, ReLU, MaxPool2d, Dropout)
- FashionMNIST를 CNN으로 분류
- FC NN vs CNN 성능/효율 비교

## 1. 합성곱(Convolution)이란?

FC 레이어는 이미지의 모든 픽셀을 1차원으로 펼쳐서 처리합니다.
이 과정에서 **인접 픽셀 간의 공간적 관계(지역성)**가 완전히 사라집니다.

합성곱은 작은 **필터(커널)**를 이미지 위에서 슬라이딩하며 특징을 추출합니다:
- **지역적 연결**: 전체가 아닌 작은 영역(예: 3×3)만 연결
- **파라미터 공유**: 동일한 필터를 이미지 전체에 재사용
- **결과**: 파라미터 수가 이미지 크기와 무관 → 대폭 감소!

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time

# FashionMNIST 데이터 준비
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

batch_size = 128
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")

## 2. CNN 핵심 구성 요소

| 레이어 | 역할 |
|---|---|
| `Conv2d` | 합성곱 연산으로 특징 추출 (커널이 이미지 위를 슬라이딩) |
| `ReLU` | 비선형성 추가 (음수 → 0, 양수 → 그대로) |
| `MaxPool2d` | 공간 크기 축소 (2×2 영역에서 최대값만 추출) |
| `Dropout` | 학습 중 일부 뉴런을 무작위로 비활성화 (과적합 방지) |
| `Linear` | 최종 분류를 위한 완전연결 레이어 |

In [ ]:
# Conv2d 동작 시각화
sample_img = training_data[0][0].unsqueeze(0)  # (1, 1, 28, 28)
print(f"입력 이미지 shape: {sample_img.shape}")

# 3×3 커널로 합성곱 적용
conv_layer = nn.Conv2d(1, 1, kernel_size=3, padding=0)
with torch.no_grad():
    conv_output = conv_layer(sample_img)
print(f"Conv2d(1→1, 3×3) 출력 shape: {conv_output.shape}")
print(f"  → 28-3+1 = 26, 크기가 약간 줄어듦")

# MaxPool2d 적용
pool_output = F.max_pool2d(conv_output, 2)
print(f"MaxPool2d(2) 출력 shape: {pool_output.shape}")
print(f"  → 26/2 = 13, 크기가 절반으로 축소")

## 3. CNN 모델 구현

```
입력 (1, 28, 28)
  → Conv2d(1→32, 3×3) → ReLU   → (32, 26, 26)
  → Conv2d(32→64, 3×3) → ReLU  → (64, 24, 24)
  → MaxPool2d(2)                → (64, 12, 12)
  → Dropout(0.25)
  → Flatten                     → (9216)
  → Linear(9216→128) → ReLU
  → Dropout(0.5)
  → Linear(128→10)
  → LogSoftmax → 출력 (10 클래스)
```

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

cnn_net = ConvNet().to(device)
print(cnn_net)

cnn_params = sum(p.numel() for p in cnn_net.parameters())
print(f"\nCNN 파라미터 수: {cnn_params:,}개")

In [ ]:
# 데이터 shape 확인
for images, labels in train_dataloader:
    print(f"배치 이미지 shape: {images.shape}")
    print(f"  → CNN은 (N, C, H, W) 형태 그대로 사용 (Flatten 불필요)")
    break

## 4. CNN 학습

In [ ]:
optimizer_cnn = torch.optim.SGD(cnn_net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

cnn_costs = []
cnn_times = []

for epoch in range(10):
    start_time = time.time()
    avg_cost = 0
    total_batch = len(train_dataloader)

    for X, Y in train_dataloader:
        X = X.to(device)  # (N, 1, 28, 28) - Flatten 하지 않음!
        Y = Y.to(device)

        optimizer_cnn.zero_grad()
        hypothesis = cnn_net(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer_cnn.step()

        avg_cost += cost.item() / total_batch

    elapsed = time.time() - start_time
    cnn_costs.append(avg_cost)
    cnn_times.append(elapsed)
    print(f"Epoch {epoch+1:02d} | cost = {avg_cost:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {sum(cnn_times)/len(cnn_times):.2f}s")

## 5. FC NN vs CNN 비교

2편에서 학습한 FC NN 결과를 재현하여 CNN과 비교합니다.

In [ ]:
# FC NN 재학습 (비교용)
class SingleLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return self.linear(x)

class MultiLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(28 * 28, 28 * 28 * 10)
        self.linear2 = nn.Linear(28 * 28 * 10, 28 * 28 * 10)
        self.linear3 = nn.Linear(28 * 28 * 10, 10)

    def forward(self, x):
        return self.linear3(self.linear2(self.linear1(x)))

# 1계층 FC 학습
fc1_net = SingleLayerNet().to(device)
fc1_opt = torch.optim.SGD(fc1_net.parameters(), lr=0.001, momentum=0.9)
fc1_costs, fc1_times = [], []

for epoch in range(10):
    start_time = time.time()
    avg_cost = 0
    for X, Y in train_dataloader:
        X, Y = X.view(-1, 784).to(device), Y.to(device)
        fc1_opt.zero_grad()
        cost = criterion(fc1_net(X), Y)
        cost.backward()
        fc1_opt.step()
        avg_cost += cost.item() / len(train_dataloader)
    fc1_costs.append(avg_cost)
    fc1_times.append(time.time() - start_time)

# 3계층 FC 학습
fc3_net = MultiLayerNet().to(device)
fc3_opt = torch.optim.SGD(fc3_net.parameters(), lr=0.001, momentum=0.9)
fc3_costs, fc3_times = [], []

for epoch in range(10):
    start_time = time.time()
    avg_cost = 0
    for X, Y in train_dataloader:
        X, Y = X.view(-1, 784).to(device), Y.to(device)
        fc3_opt.zero_grad()
        cost = criterion(fc3_net(X), Y)
        cost.backward()
        fc3_opt.step()
        avg_cost += cost.item() / len(train_dataloader)
    fc3_costs.append(avg_cost)
    fc3_times.append(time.time() - start_time)

print("FC 1-Layer / FC 3-Layer / CNN 학습 완료")

In [ ]:
fc1_params = sum(p.numel() for p in fc1_net.parameters())
fc3_params = sum(p.numel() for p in fc3_net.parameters())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
epochs = range(1, 11)

# Cost 비교
axes[0].plot(epochs, fc1_costs, "o-", label=f"FC 1-Layer ({fc1_params:,})")
axes[0].plot(epochs, fc3_costs, "s-", label=f"FC 3-Layer ({fc3_params:,})")
axes[0].plot(epochs, cnn_costs, "^-", label=f"CNN ({cnn_params:,})")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Cost")
axes[0].set_title("Cost Comparison")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 시간 비교
axes[1].plot(epochs, fc1_times, "o-", label="FC 1-Layer")
axes[1].plot(epochs, fc3_times, "s-", label="FC 3-Layer")
axes[1].plot(epochs, cnn_times, "^-", label="CNN")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Time (seconds)")
axes[1].set_title("Training Time per Epoch")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 결과 요약 테이블
print(f"{'모델':15s} {'파라미터':>15s} {'평균 시간(s)':>12s} {'최종 Cost':>12s}")
print("-" * 58)
print(f"{'FC 1-Layer':15s} {fc1_params:>15,} {sum(fc1_times)/10:>12.2f} {fc1_costs[-1]:>12.6f}")
print(f"{'FC 3-Layer':15s} {fc3_params:>15,} {sum(fc3_times)/10:>12.2f} {fc3_costs[-1]:>12.6f}")
print(f"{'CNN':15s} {cnn_params:>15,} {sum(cnn_times)/10:>12.2f} {cnn_costs[-1]:>12.6f}")
print()
print(f"CNN은 FC 3-Layer 대비:")
print(f"  파라미터: {cnn_params/fc3_params*100:.1f}% (약 {fc3_params/cnn_params:.0f}배 적음)")
print(f"  학습시간: {sum(cnn_times)/sum(fc3_times)*100:.0f}%")

## CNN이 효과적인 이유

| FC (완전연결) | CNN (합성곱) |
|---|---|
| 모든 픽셀을 1차원으로 펼침 | 2D 공간 구조를 유지 |
| 모든 입출력이 연결 | 커널 크기만큼만 지역 연결 |
| 파라미터 = 입력 × 출력 | 파라미터 = 커널 크기 × 채널 수 |
| 이미지 크기에 비례하여 폭발 | 이미지 크기와 무관 |

핵심 아이디어: **이미지는 지역적 패턴의 조합**이다.
- 저수준 특징: 엣지, 코너, 텍스처
- 고수준 특징: 형태, 패턴, 객체
- CNN은 이 계층 구조를 자연스럽게 학습

→ **다음 노트북에서 GAN(생성적 적대 신경망)으로 이미지를 '생성'하는 방법을 학습합니다.**